# Getting Started with GraphRAG-SDK

This notebook provides an introduction to using FalkorDB's GraphRAG software development kit.It guides users through the setup process, dataset loading, and running basic examples.Ideal for new users, this notebook helps familiarize with core functionalities and workflowsin GraphRAG for developing graph-based applications and analyses.

Install dependencies

In [ ]:
! python --version

Python 3.10.12


In [ ]:
# prompt: pip install FalkorDB & GraphRAG-SDK

!pip install FalkorDB
!pip install graphrag_sdk==0.1.3b0

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.5/327.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.8 MB/s eta 0:00:00
  Created wheel for graphrag_sdk: filename=graphrag_sdk-0.1.3b0-py3-none-any.whl size=26462 sha256=69ae6d4408a40b6cc4119e670a7a82af70225c0f9d3c412426aad4c798b40732
  Stored in directory: /root/.cache/pip/wheels/b5/e7/44/c5997105d448c1b73d27b9b2ae440afafc1617e495f9c7fb0e
Successfully built graphrag_sdk


Setup dependencies:

1. OpenAI KEY

2. FalkorDB connection details

Create a FalkorDB instance at: http://cloud.falkordb.com/

And update the following fields accordingly: host, port, username, password

In [ ]:
# set OpenAI key
import os
os.environ["OPENAI_API_KEY"] = "YOUR-OPENAI-KEY"

# Define connection details for FalkorDB
host = "FALKORDB_HOST"
port = "FALKORDB_PORT"
username = 'FALKORDB_USERNAME'
password = 'FALKORDB_PASSWORD'

Auto detect knowledge graph ontology

In [ ]:
from graphrag_sdk import Source
from graphrag_sdk.schema import Schema

# Auto detect schema based on a single URL
source = Source("https://www.rottentomatoes.com/m/matrix")
s = Schema.auto_detect([source])

# Print schema
print("Auto detected schema:")
print(s.to_JSON())

Auto detected schema:
{
    "entities": [
        {
            "name": "Reviews",
            "attributes": [
                {
                    "name": "review_date",
                    "type": "str",
                    "desc": "Reviews's review_date",
                    "unique": true,
                    "mandatory": true
                },
                {
                    "name": "movie_title",
                    "type": "str",
                    "desc": "Reviews's movie_title",
                    "unique": false,
                    "mandatory": false
                },
                {
                    "name": "reviewer",
                    "type": "str",
                    "desc": "Reviews's reviewer",
                    "unique": false,
                    "mandatory": false
                },
                {
                    "name": "rating",
                    "type": "str",
                    "desc": "Reviews's rating",
                    "uniqu

Manually define knowledge graph ontology

In [ ]:
# Manually define schema
s = Schema()

# Entities:
# 1. Actor
# 2. Director
# 3. Movie

actor = s.add_entity('Actor')
actor.add_attribute('Name', str, unique=True, mandatory=True)

director = s.add_entity('Director')
director.add_attribute('Name', str, unique=True, mandatory=True)

movie = s.add_entity('Movie')
movie.add_attribute('Title', str, unique=True, mandatory=True)
movie.add_attribute('ReleaseYear', int)
movie.add_attribute('Rating', int)

# Relations:
# 1. (Actor)-[ACTED]->(Movie)
# 2. (Director)-[DIRECTED]->(Movie)

s.add_relation("ACTED", actor, movie)
s.add_relation("DIRECTED", director, movie)

# Print schema
print("Manually defined schema:")
print(s.to_JSON())

Manually defined schema:
{
    "entities": [
        {
            "name": "Movie",
            "attributes": [
                {
                    "name": "ReleaseYear",
                    "type": "int",
                    "desc": "Movie's ReleaseYear",
                    "unique": false,
                    "mandatory": false
                },
                {
                    "name": "Title",
                    "type": "str",
                    "desc": "Movie's Title",
                    "unique": true,
                    "mandatory": true
                },
                {
                    "name": "Rating",
                    "type": "int",
                    "desc": "Movie's Rating",
                    "unique": false,
                    "mandatory": false
                }
            ]
        },
        {
            "name": "Actor",
            "attributes": [
                {
                    "name": "Name",
                    "type": "str",
      

Construct knowledge graph
From a number of sources:
1. https://www.rottentomatoes.com/m/matrix
2. https://www.rottentomatoes.com/m/matrix_reloaded
3. https://www.rottentomatoes.com/m/the_matrix_resurrections

In [ ]:
from graphrag_sdk import KnowledgeGraph
# Create Knowledge Graph
g = KnowledgeGraph("rottentomatoes", host=host, port=port, username=username,
                   password=password, schema=s, model="gpt-3.5-turbo-0125")

# Ingest
# Define sources from which knowledge will be created
sources = [
    Source("https://www.rottentomatoes.com/m/matrix"),
    Source("https://www.rottentomatoes.com/m/matrix_reloaded"),
    Source("https://www.rottentomatoes.com/m/the_matrix_resurrections")
]

g.process_sources(sources)

Predefined questions

In [ ]:
# Query
msgs = []
questions = [
    "List a few actors",
    "Provide a break down of the number of movies each actored played in",
    "Which movie has the heighest Rating?"
]

for q in questions:
  print(f"Question: {q}")
  ans, msgs = g.ask(q, history=msgs)
  print(f"Answer: {ans}\n")

Question: List a few actors
Answer: Here are a few actors:
1. Keanu Reeves
2. Carrie-Anne Moss
3. Yahya Abdul-Mateen II
4. Jonathan Groff
5. Jessica Henwick

Question: Provide a break down of the number of movies each actored played in
Answer: Here is the breakdown of the number of movies each actor played in:
- Keanu Reeves: 2 movies
- Jessica Henwick: 1 movie
- Carrie-Anne Moss: 2 movies
- Laurence Fishburne: 1 movie
- Jonathan Groff: 1 movie
- Yahya Abdul-Mateen II: 1 movie

Question: Which movie has the heighest Rating?
Answer: The movie with the highest rating is "The Matrix" with a rating of 83.



Q&A Session
Type exit to quit.

In [ ]:
while True:
  q = input("Question: ")
  if q == "exit":
    break

  ans, msgs = g.ask(q, history=msgs)
  print(f"Answer: {ans}\n")

Question: Which movies are in the dataset ?
Answer: The movies in the dataset are:
1. The Matrix
2. The Matrix Resurrections
3. The Matrix Reloaded

Question: Which movie has the largest cast ?
Answer: The movie with the largest cast is "The Matrix Resurrections" with a cast size of 5.

Question: exit
